In [12]:
# ! pip install transformers==4.30.1
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
adapter-transformers 3.2.1 requires tokenizers!=0.11.3,<0.14,>=0.11.1, but you have tokenizers 0.15.2 which is incompatible.


In [13]:
# ! pip install adapter-transformers==3.2.1

In [14]:
!pip install bitsandbytes
!pip install accelerate

In [1]:
import numpy as np
from transformers import TrainingArguments, Trainer, EvalPrediction

In [2]:
import transformers
import torch

In [3]:
torch.cuda.is_available()

True

In [4]:
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
import pandas as pd
from google.colab import files

In [7]:
uploaded = files.upload()

Saving consumer_complaints_5k.csv to consumer_complaints_5k (1).csv


In [8]:
import io
try:
  dataset = pd.read_csv('emails.csv')
except:
  # dataset = pd.read_excel(io.BytesIO(uploaded['naics_desc_data_2022.xlsx']))
  dataset = pd.read_csv("consumer_complaints_5k.csv")

In [9]:
dataset.head()

,Unnamed: 0,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,48,2024-02-17,Credit card,General-purpose credit card or charge card,Problem with a purchase shown on your statement,Credit card company isn't resolving a dispute ...,"I paid {$1000.00} to have artwork restored, ma...",Company has responded to the consumer and the ...,"CITIBANK, N.A.",LA,701XX,NaN,Consent provided,Web,2024-02-17,Closed with monetary relief,Yes,NaN,8359230
1,67,2024-02-16,"Money transfer, virtual currency, or money ser...",Domestic (US) money transfer,Fraud or scam,NaN,Back in XXXX of 2023 I received a call from so...,Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,TX,779XX,NaN,Consent provided,Web,2024-02-16,Closed with explanation,Yes,NaN,8360700
2,81,2023-12-11,Student loan,Private student loan,Getting a loan,Loan opened without my consent or knowledge,Name : XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX...,NaN,"Nelnet, Inc.",IL,60640,NaN,Consent provided,Web,2023-12-11,Closed with explanation,Yes,NaN,7980844
3,83,2024-02-17,Credit reporting or other personal consumer re...,Credit reporting,Improper use of your report,Reporting company used your report improperly,Subject : Cease and Desist Letter Regarding Un...,Company has responded to the consumer and the ...,PENTAGON FEDERAL CREDIT UNION,NJ,07753,NaN,Consent provided,Web,2024-02-17,Closed with explanation,Yes,NaN,8367075
4,132,2024-02-13,Checking or savings account,Checking account,Opening an account,Didn't receive terms that were advertised,"Hello, I opened a new account with Wells Fargo...",Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,CA,92648,NaN,Consent provided,Web,2024-02-13,Closed with monetary relief,Yes,NaN,8337555


In [10]:
dataset.columns

Index(['Unnamed: 0', 'Date received', 'Product', 'Sub-product', 'Issue',
       'Sub-issue', 'Consumer complaint narrative', 'Company public response',
       'Company', 'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID'],
      dtype='object')

In [11]:
# dataset = dataset[:5000]

In [12]:
sentences = dataset['Consumer complaint narrative'].values
labels = dataset['Product'].values

In [13]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(labels)

LabelEncoder()

In [14]:
y_enc = le.transform(labels)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sentences, y_enc, test_size=0.25, random_state=42)

In [16]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)

Loading BERT tokenizer...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [17]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

df1 = pd.DataFrame({'sentences': list(X_train), 'labels': list(y_train)})
dataset = ds.dataset(pa.Table.from_pandas(df1).to_batches())

### convert to Huggingface dataset
hg_dataset1 = Dataset(pa.Table.from_pandas(df1))

In [18]:
df2 = pd.DataFrame({'sentences': list(X_test), 'labels': list(y_test)})
dataset = ds.dataset(pa.Table.from_pandas(df2).to_batches())

### convert to Huggingface dataset
hg_dataset2 = Dataset(pa.Table.from_pandas(df2))

In [19]:
import datasets
dd = datasets.DatasetDict({"train":hg_dataset1,"test":hg_dataset2})

In [20]:
def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(
      batch["sentences"],
      max_length=256,
      truncation=True,
      padding="max_length"
  )

# Encode the input data
dd = dd.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
# dd = dd.rename_column("labels", "labels")
# Transform to pytorch tensors and only output the required columns
dd.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/3750 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

In [21]:
from transformers import BertConfig, BertModelWithHeads

id2label = dict(list(enumerate([str(x) for x in le.classes_])))

config = BertConfig.from_pretrained(
    "bert-large-uncased",
    id2label=id2label,
)
model = BertModelWithHeads.from_pretrained(
    "bert-large-uncased",
    config=config,
)


/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/bert/adapter_model.py:269: FutureWarning: This class has been renamed to `BertAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/bert/adapter_model.py:247: FutureWarning: This class has been renamed to `BertAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModelWithHeads: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModelWithHeads from the checkpoint of a model that you expect to be exactl

In [22]:
model.add_classification_head("cb", num_labels=len(id2label))
model.to(device)

BertModelWithHeads(
  (shared_parameters): ModuleDict()
  (bert): BertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(
                in_features=1024, out_features=1024, bias=True
                (loras): ModuleDict()
              )
              (key): Linear(
                in_features=1024, out_features=1024, bias=True
                (loras): ModuleDict()
              )
              (value): Linear(
                in_features=1024

In [23]:
import numpy as np
from transformers import TrainingArguments, Trainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=5e-5,
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=200,
    output_dir="./training_output",
    # overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dd["train"],
    eval_dataset=dd["test"],
    compute_metrics=compute_accuracy,
)

In [24]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3750
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 940
  Number of trainable parameters = 336203788


Step,Training Loss
200,0.864500
400,0.569200
600,0.397100
800,0.270500


Saving model checkpoint to ./training_output/checkpoint-500
Configuration saved in ./training_output/checkpoint-500/config.json
Configuration saved in ./training_output/checkpoint-500/generation_config.json
Model weights saved in ./training_output/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=940, training_loss=0.4749347463567206, metrics={'train_runtime': 2212.6589, 'train_samples_per_second': 6.779, 'train_steps_per_second': 0.425, 'total_flos': 7013904168960000.0, 'train_loss': 0.4749347463567206, 'epoch': 4.0})

In [25]:
# trainer.save_model("path_to_save")
trainer.save_model("bert_large_cfpb_5k")

Saving model checkpoint to bert_large_cfpb_5k
Configuration saved in bert_large_cfpb_5k/config.json
Configuration saved in bert_large_cfpb_5k/generation_config.json
Model weights saved in bert_large_cfpb_5k/pytorch_model.bin


In [26]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1250
  Batch size = 16


{'eval_runtime': 62.8381,
 'eval_samples_per_second': 19.892,
 'eval_steps_per_second': 1.257,
 'epoch': 4.0}

In [27]:
preds=trainer.predict(dd["test"])
(np.argmax(preds.predictions[1],axis=1) == dd["test"]["labels"].numpy()).mean()

***** Running Prediction *****
  Num examples = 1250
  Batch size = 16


0.8536

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(dd["test"]["labels"].numpy(),np.argmax(preds.predictions[1],axis=1)))

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
# See https://www.reddit.com/r/StableDiffusion/comments/ynopq6/how_do_i_download_my_trained_model_from_google/
!cp -r "/content/bert_large_cfpb_5k" "/content/drive/MyDrive"

## Quantization

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [11]:
# See https://github.com/TimDettmers/bitsandbytes/issues/428
# tokenizer = AutoTokenizer.from_pretrained("/content/bert_large_cfpb_5k")
model_q = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/bert_large_cfpb_5k", load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/bert_large_cfpb_5k")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at /content/drive/MyDrive/bert_large_cfpb_5k and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at /content

In [12]:
print(model.num_parameters(), model_q.num_parameters())

335174458 335204980


In [13]:
def find_model_size(model):
  param_size = 0
  for param in model.parameters():
      param_size += param.nelement() * param.element_size()
  buffer_size = 0
  for buffer in model.buffers():
      buffer_size += buffer.nelement() * buffer.element_size()

  size_all_mb = (param_size + buffer_size) / 1024**2
  print('model size: {:.3f}MB'.format(size_all_mb))

In [14]:
print(find_model_size(model), find_model_size(model_q))

model size: 639.302MB
model size: 205.861MB
None None
